# 욕설 분류기
---------
## CNN을 이용한 인터넷 채팅상의 욕설 분류 

### 사용패키지

In [2]:
from konlpy.tag import Kkma
import hgtk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
import tensorflow as tf

----------------
## 1) 전처리
### 사용할 초성, 중성, 종성 리스트 합본

In [4]:
kkma = Kkma()

In [5]:
jong_list = hgtk.const.JONG[1:]
jamo = (hgtk.const.CHO,hgtk.const.JOONG,tuple(jong_list))

In [6]:
def process_sentence(raw_sentence):
    noun_and_unknown = []
    for word in kkma.pos(raw_sentence):
        if word[1] == "NNG" or word[1] == "UN":
            noun_and_unknown.append(word[0])
    decompose_sentence = "".join(noun_and_unknown)
    count = len(decompose_sentence)
    range_list = [(0,5)]
    if count > 5:
        i = 0
        while (i+5) < count:
            i += 2
            range_list.append((i,i+5))
    decom_list = []
    for s,t in range_list:
        decom_list.append(hgtk.text.decompose(decompose_sentence[s:t]))
    return decom_list, len(decom_list)

a,b= process_sentence("이 씹새끼가 어디서 약을 팔어 씹ㅏㄹ새끼야")  
print(a,b)

------
['ㅆㅣㅂᴥㅅㅐᴥㄲㅣᴥㅇㅑㄱᴥㅍㅏㄹᴥ',  
'ㄲㅣᴥㅇㅑㄱᴥㅍㅏㄹᴥㅆㅣㅂᴥㅏᴥ',  
'ㅍㅏㄹᴥㅆㅣㅂᴥㅏᴥㄹᴥㅅㅐᴥ',  
'ㅏᴥㄹᴥㅅㅐᴥㄲㅣᴥ'] 4

In [7]:
def let2mat(processed):
    result_np = np.zeros((5,3))
    i, index = 0, 0
    for c in processed:
        if c == 'ᴥ':
            index += 1
            i = 0
            continue
        if (c not in jamo[i]):
            i += 1
        result_np[index,i] = jamo[i].index(c) + 1 #공백과 첫번째 인덱스를 구분하기 위해서
        i += 1
    return result_np

In [8]:
def process(raw_sentence):
    a,b = process_sentence(raw_sentence)
    result = let2mat(a[0])
    for i in range(1,b):
        if result.ndim == 2:
            result = np.r_[[result],[let2mat(a[i])]]
            continue
        result = np.r_[result,[let2mat(a[i])]]
    return result

a,b = process_sentence("이 씹새끼가 어디서 약을 팔어 씹ㅏㄹ새끼야")
print(a,b)
process("이 씹새끼가 어디서 약을 팔어 씹ㅏㄹ새끼야")

-----
['ㅆㅣㅂᴥㅅㅐᴥㄲㅣᴥㅇㅑㄱᴥㅍㅏㄹᴥ', 'ㄲㅣᴥㅇㅑㄱᴥㅍㅏㄹᴥㅆㅣㅂᴥㅏᴥ', 'ㅍㅏㄹᴥㅆㅣㅂᴥㅏᴥㄹᴥㅅㅐᴥ', 'ㅏᴥㄹᴥㅅㅐᴥㄲㅣᴥ'] 4  

      [[[11., 21., 17.],  
        [10.,  2.,  0.],  
        [ 2., 21.,  0.],  
        [12.,  3.,  1.],  
        [18.,  1.,  8.]],  
        
       [[ 2., 21.,  0.],  
        [12.,  3.,  1.],  
        [18.,  1.,  8.],  
        [11., 21., 17.],  
        [ 0.,  1.,  0.]],  
        
       [[18.,  1.,  8.],  
        [11., 21., 17.],  
        [ 0.,  1.,  0.],  
        [ 6.,  0.,  0.],  
        [10.,  2.,  0.]],  
        
       [[ 0.,  1.,  0.],  
        [ 6.,  0.,  0.],  
        [10.,  2.,  0.],  
        [ 2., 21.,  0.],  
        [ 0.,  0.,  0.]]]